In [15]:
import os
print("Current Working Directory " , os.getcwd())

Current Working Directory  /Users/leducanh/Documents/pythonProject


In [54]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd


class Scraper:
    def __init__(self, mainUrl):
        self.links = {
            'categories': [],
            'pages': {},
            'jobOffers': []
        }
        self.data = {
            'jobTitles': [],
            'company': [],
            'salaryRange': [],
            'afterBonusSal': [],
            'category+progLang': [],
            'seniority': [],
            'workLanguage': [],
            'location': [],
            'remote':[],
            'postingTime':[]
        }
        self.mainUrl = mainUrl
        self.categList = []

    # getting the links for job categories
    def getUrlByCat(self, url):

        response = requests.get(url)

        soup = BeautifulSoup(response.text, 'lxml')
        linksByCat = soup.find_all('div',
                                   class_="d-flex justify-content-between align-items-center list-title-wrapper ng-star-inserted")
        urlList = [url + href.a['href'] for href in linksByCat]
        # removing redundant links in urlList
        self.links['categories'] = [link for link in urlList if '?criteria' not in link]

        #get a list of job categories, which will be used as keys in the links['pages'], links['jobOffers'] dictionary
        self.categList = [self.links['categories'][i].rsplit('/', 1)[-1] for i in range(len(self.links['categories']))]
        print('categorical urls success')

    def getPages(self):
        ''' fill the self.links['pages'], where categories are denoted as keys with the value being a list containing
                 the link to the first page of job offers of the particular category
                 later, the idea will be to append these lists, with the links to the next job offer pages'''

        self.links['pages'] = {k: [v] for v, k in zip(self.links['categories'], self.categList)}

        '''e.g. links['pages'] = {'backend': ['https://nofluffjobs.com/cz/it-prace/backend'], 
                                  'frontend': ['https://nofluffjobs.com/cz/it-prace/frontend'],
                                   ...}'''

        # adding the page url from the next button (first url page is already added)
        for url in self.links['pages'].values():
            while True:
                try:
                    response = requests.get(url[-1])
                    soup = BeautifulSoup(response.text, 'lxml')
                    pageLink = self.mainUrl + soup.find('a', {'aria-label': 'Next'}).get('href')
                    url.append(pageLink)
                except AttributeError:
                    break
                time.sleep(0.2)
                response = requests.get(pageLink)
                #soup = BeautifulSoup(response.text, 'lxml')
        print('page urls success')

    def getJobLinks(self):
        #add keys based on job categories to the self.links['jobOffers'] dictionary  ,the values are emtpy lists
        #self.links['jobOffers'] = {k: [] for k in self.categList}
        #pageLinks is a dictionary with key: list values, where key is the job category and lists contain links for the pages
        for lists in self.links['pages'].values():
            for link in lists:
                time.sleep(0.1)
                # request applied on a particular page
                response = requests.get(link).text
                soup = BeautifulSoup(response, 'lxml')
                #select all the 'a' tags on the particular page with the idea to capture all the 'hrefs' from there
                aTags = soup.select(
                    'body > nfj-root > nfj-layout > nfj-main-content > div > div.main-content__outlet.mb-5.pb-5 >nfj-postings-search > div > common-main-loader > div > nfj-search-results > nfj-postings-list:nth-child(1) >div.list-container.ng-star-inserted>a')

                #looping through the 'a' tags to capture the 'hrefs' which are than added into the 'jobOffers' dictionary's list
                #keys denotes the job category, where the particular 'href' belongs to
                for href in aTags:
                    self.links['jobOffers'].append(self.mainUrl + href.get('href'))
        print('job sites url success')
    def getData(self):
        for link in self.links['jobOffers']:    
            time.sleep(0.2)
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'lxml')
            #getting job titles
            jobTitle = soup.find('div',
                                 class_="posting-details-description d-flex align-items-center align-items-lg-start flex-column justify-content-center justify-content-lg-start")
            try:
                self.data['jobTitles'].append(jobTitle.h1.text)
            except:
                self.data['jobTitles'].append(None)
            self.data['company'].append(jobTitle.a.text)

            #getting offered salary ranges
            salaryRange = soup.find('div', class_="salary")
            self.data['salaryRange'].append(salaryRange.h4.text)

            #getting salaries after bonus
            bonus = soup.find('span', class_="bonus-value color-main")
            try:
                self.data['afterBonusSal'].append(bonus.text)
            # afterBonusSal equals to salaryRange
            except:
                self.data['afterBonusSal'].append(salaryRange.h4.text)

            #getting job's location
            location = soup.find('common-posting-locations')
            try:
                self.data['location'].append(location.text)
            except:
                self.data['location'].append(None)
            
            #getting the info, whether it is a remote job
            remote = soup.find('li', class_="remote")
            try:
                self.data['remote'].append(remote.text)
            except:
                self.data['remote'].append(None)
            
            #getting info on the posting time of the job offer
            postingTime = soup.find('div', class_= 'posting-time-row')
            self.data['postingTime'].append(postingTime.text)
            

            #getting offered job category (type of job + the main language
            #we will get 2 divs from the find_all, each div representing a row on the website
            basicInfos = soup.find_all('div',
                                        class_="posting-info-row d-flex flex-column flex-md-row justify-content-between justify-content-center align-items-md-center")

            for index, info in enumerate(basicInfos):
                # if we are scraping from the upper row
                if index % 2 == 0:
                    # getting job category+language or work language (=english or czech)
                    spanBasicInfo = info.find('span', class_='font-weight-semi-bold')
                    self.data['category+progLang'].append(spanBasicInfo.text)

                    spanSenior = info.find('span', class_="mr-10 font-weight-medium")
                    self.data['seniority'].append(spanSenior.text)
                else:
                    try:
                        spanWorkLang = info.find('span', class_='font-weight-semi-bold')
                        self.data['workLanguage'].append(spanWorkLang.text)
                    except:
                        self.data['workLanguage'].append(None)
        print('data extraction success')

In [59]:
mainUrl = 'https://nofluffjobs.com'

site = Scraper(mainUrl)

site.getUrlByCat(mainUrl)

site.getPages()

site.getJobLinks()

site.getData()




categorical urls success
page urls success
job sites url success
data extraction success


In [63]:
data = pd.DataFrame(site.data)
data.head()

,jobTitles,company,salaryRange,afterBonusSal,category+progLang,seniority,workLanguage,location,remote,postingTime
0,"Java Developer - zdravotnictví, letectví",Randstad HR Solutions s.r.o.,70k - 90k CZK,70k - 90k CZK,"Backend, java",Mid,česky,"Praha, Plzeň • Praha• Plzeň",None,Tato nabídka byla zveřejněna dnes
1,Solution Architect,Randstad HR Solutions s.r.o.,100k - 150k CZK,100k - 150k CZK,"Backend, Enterprise Architect","Mid, Senior",česky,Praha • Praha,None,Tato nabídka byla zveřejněna dnes
2,C++ Embedded - automotive,Randstad HR Solutions s.r.o.,75k - 100k CZK,75k - 100k CZK,"Backend, c++",Mid,"česky,",Praha • Praha,None,Tato nabídka byla zveřejněna dnes
3,Lead Engineer Node.js,OAK'S LAB,75k - 150k CZK,75k - 150k CZK,"Backend, node",Senior,česky,"Prague, Old Town Square 604/14 (Po pandemi)• ...",Covid-time na dálku,Tato nabídka byla zveřejněna dnes
4,Backend Engineer (Node.js),OAK'S LAB,70k - 140k CZK,70k - 140k CZK,"Backend, NodeJS",Mid,česky,"Prague, Old Town Square 604/14 (Po pandemi)• ...",Covid-time na dálku,Tato nabídka byla zveřejněna dnes


In [61]:
data.to_csv('jobOffers.csv')